In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/workspaces/stok_opt-mimizasyon_mlops_project'

## 1.ENTİTİY

Değişkenlerimi ve değişkenlerimin tiplerini oluşturacağım


In [4]:
from dataclasses import dataclass
from pathlib import Path
from src.stok_optimizasyonu_ml_project.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.stok_optimizasyonu_ml_project.utils.common import read_yaml,create_directories

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# CONFİG
Yukarıda değişkenlerini tanımladığımız yapıların değerlerini verelim

In [5]:
from src.stok_optimizasyonu_ml_project.constants import *
from src.stok_optimizasyonu_ml_project.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        print("Yüklenen config:", self.config)  # Debug için eklendi
        
        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: # Yukarida DataIngestionConfig sinifi icerisinde tanimlamis oldugum degiskenleri return edecektir
        config = self.config.data_ingestion # root_dir, local_data_file, source_URL, unzip_dir keylerine erisim sagliyorum

        create_directories([config.root_dir]) # artifacts/data_ingestion isimli bir klasor olustuyorum

        data_ingestion_config = DataIngestionConfig( # Ust hucrede tanimlamis oldugum sinifin nesnesini yaratiyorum
            root_dir=config.root_dir, #artifacts/data_ingestion
            source_URL=config.source_URL, #https://www.kaggle.com/api/v1/datasets/download/bhavikjikadara/student-study-performance
            local_data_file=config.local_data_file, # artifacts/data_ingestion/data.zip
            unzip_dir=config.unzip_dir, # artifacts/data_ingestion
        )

        return data_ingestion_config

# COMPONENTS

In [7]:
import os
import urllib.request as request
import zipfile
from src.stok_optimizasyonu_ml_project import logger
from src.stok_optimizasyonu_ml_project.utils.common import get_size

In [8]:

class DataIngestion:
    def __init__(self,config: DataIngestionConfig): # config parametresi DataIngestionConfig icerisnde olusturulmus degiskenlerini ve tiplerini almasi saglaniyor
        self.config = config # config degiskenine deger olarak DataIngestion sinifindan nesne olusturken config parametresinin yerine ne deger ataniyorsa aynisi config degiskenine atanmasi saglaniyor


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): # data.zip isimli bir dosya yoksa True Donecektir
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, # url parametre dosyanin hangi siteden indirilmesi gerektigini alir
                filename = self.config.local_data_file # dosyayi hangi isimle locale indirmen gerektigini yazar
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else: # eger onceden boyle bir dosya varsa
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  #data.zip dosyasinin kb olarak ne kadar buyuk oldugunu loglar


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir # unzip_path degiskenine artifacts/data_ingestion bu degeri atiyoruz
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref: # zip_ref adini verdigimiz bir degiskenle data.zipin icerini acip okuyoruz
            zip_ref.extractall(unzip_path) # extractall metoduyla dosyami zipten cikarip artifacts/data_ingestion bu klasor icerisine atiyorum

    

# PİPELİNE

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

Yüklenen config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://www.kaggle.com/api/v1/datasets/download/zkankurukavak/satslar-zkan', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/satslar.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation/', 'data_path': 'artifacts/data_ingestion/satslar.csv', 'transformed_data_path': 'artifacts/data_transformation/transformed_data.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'xgboost', 'model_save_path': 'artifacts/model_trainer/model.joblib', 'grid_search': True, 'scoring_metric': 'neg_mean_squared_erro

TypeError: Subscripted generics cannot be used with class and instance checks